In [2]:
import pandas as pd
from operator import add

In [3]:
# Reading the client file
df = pd.read_csv('data_with_clientid.csv')

In [4]:
df.head()

,id,client_id,first_name,last_name,email,country,role,language,total_logins,account_status,...,score,completion_percent,learning_skills_listening,learning_skills_writing,learning_skills_speaking,learning_skills_reading,lesson_id,progress_data,sco_number,lesson_attempts
0,271559,394,Ximena,Espitia,a54340997901aaac95c893e5b174201d72941c64e83455...,Canada,student,Spanish,4,active,...,83,1,60,40,0,59,11493,"{""at_score"": ""-1"", ""datamodel"": {""cmi"": {""core...",7225,1
1,271558,394,Luis David,Salcedo,99f6ca57e808f286d726496455ee5006e4e6045f9fba71...,Canada,student,Spanish,3,active,...,89,1,94,100,100,93,11493,"{""at_score"": ""-1"", ""datamodel"": {""cmi"": {""core...",7225,1
2,271558,394,Luis David,Salcedo,99f6ca57e808f286d726496455ee5006e4e6045f9fba71...,Canada,student,Spanish,3,active,...,89,1,94,100,100,93,11494,"{""at_score"": ""-1"", ""datamodel"": {""cmi"": {""core...",7226,1
3,271558,394,Luis David,Salcedo,99f6ca57e808f286d726496455ee5006e4e6045f9fba71...,Canada,student,Spanish,3,active,...,89,1,94,100,100,93,11495,"{""at_score"": ""-1"", ""datamodel"": {""cmi"": {""core...",7227,1
4,271558,394,Luis David,Salcedo,99f6ca57e808f286d726496455ee5006e4e6045f9fba71...,Canada,student,Spanish,3,active,...,89,1,94,100,100,93,11496,"{""at_score"": ""-1"", ""datamodel"": {""cmi"": {""core...",7228,1


In [5]:
df.shape

(10000, 21)

In [6]:
# Calculating the average score by country 
df.groupby('country', as_index=False)['score'].mean()

,country,score
0,Brazil,76.250000
1,Canada,56.489796
2,Chile,64.000000
3,Colombia,86.266112
4,Mexico,81.776316
5,Peru,84.648620
6,Spain,72.625000


In [7]:
# Calculating the average score by language
df.groupby('language', as_index=False)['score'].mean()

,language,score
0,English,87.458007
1,Portuguese,81.624088
2,Spanish,85.050944


In [8]:
# Calculating the average of all the four skills
group_by_information = df.groupby('id', as_index=False)['learning_skills_listening', 'learning_skills_reading', 'learning_skills_speaking', 'learning_skills_writing', 'score'].mean()

/var/folders/98/sznc5p5954z41vgg3v246x500000gn/T/ipykernel_46722/2618720962.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group_by_information = df.groupby('id', as_index=False)['learning_skills_listening', 'learning_skills_reading', 'learning_skills_speaking', 'learning_skills_writing', 'score'].mean()


In [9]:
group_by_information

,id,learning_skills_listening,learning_skills_reading,learning_skills_speaking,learning_skills_writing,score
0,264175,88.0,81.0,52.0,56.0,49.0
1,264176,97.0,96.0,87.0,88.0,85.0
2,264177,99.0,98.0,95.0,94.0,78.0
3,264178,100.0,100.0,99.0,98.0,97.0
4,264179,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...
817,271153,76.0,83.0,94.0,87.0,78.0
818,271160,88.0,88.0,100.0,100.0,80.0
819,271537,64.0,78.0,79.0,52.0,76.0
820,271558,94.0,93.0,100.0,100.0,89.0


In [10]:
def add_lists(l1, l2):
    result = []
    for i in range(len(l1)):
        result.append(l1[i] + l2[i])
    return result

In [11]:
import json

# Method to compute the listening, reading, writing score after parsing the json
# +1 for any if the student passed and the skill was True
# -1 for any if the student not passed/failed and skill was True

def parse_json(progress_data):
    json_input = json.loads(progress_data)
    items = json_input['datamodel']['cmi']['objectives']['items']
    
    listening, reading, writing, speaking = 0,0,0,0

    for item in items:
        try:
            if(item['status'] == 'failed' or item['status'] == 'not passed'):
                if(item['listening'] == True):
                    listening -= 1
                if(item['reading'] == True):
                    reading -= 1
                if(item['writing'] == True):
                    writing -= 1
                if(item['speaking'] == True):
                    speaking -= 1
            else:
                if(item['listening'] == True):
                    listening += 1
                if(item['reading'] == True):
                    reading += 1
                if(item['writing'] == True):
                    writing += 1
                if(item['speaking'] == True):
                    speaking += 1
        except:
            continue

    return [listening, reading, writing, speaking]

In [12]:
def process_json(df):
    progress_data = df[['id', 'progress_data']]
    user_info = {}
    for index, row in df.iterrows():
        if(row['id'] not in user_info):
            user_info[row['id']] = parse_json(row['progress_data'])
        else:
            skills = user_info[row['id']]
            result = parse_json(row['progress_data'])
            user_info[row['id']] = add_lists(skills, result)
    return user_info

In [13]:
user_info = process_json(df)
user_info

{271559: [-2, -2, -2, -1],
 271558: [9, 19, 6, -1],
 271537: [-6, -4, -2, -3],
 271160: [4, 10, 4, 1],
 271153: [-4, 2, -2, -3],
 271152: [-2, 0, -2, 1],
 271149: [37, 89, 27, 22],
 271148: [19, 18, 12, -10],
 271147: [10, 36, 7, 7],
 271146: [5, 5, 1, 2],
 271145: [10, 27, 11, 14],
 271127: [14, 28, 13, 7],
 271119: [0, 4, -2, 1],
 271117: [6, 8, 2, 3],
 271113: [30, 52, 12, 7],
 271110: [3, 3, 1, 2],
 271105: [-2, -4, 0, -1],
 271103: [2, 10, -2, 3],
 271099: [-4, -6, -2, -1],
 271096: [-3, -1, -5, 0],
 271094: [15, 28, 10, 8],
 271089: [3, 4, 3, 1],
 271082: [-4, -6, -2, -1],
 271081: [2, 5, 2, 2],
 271080: [-2, -1, -1, -1],
 271077: [4, 15, -2, 0],
 271072: [0, 1, 1, -1],
 271071: [1, 4, 0, 1],
 271068: [-6, -9, -6, 0],
 271066: [3, 6, 2, 3],
 271065: [-2, -3, -1, -1],
 271063: [6, 13, 4, 4],
 270993: [5, 26, 9, 2],
 270992: [1, 8, 2, 2],
 270991: [3, 4, 2, 2],
 270988: [3, 14, 2, 1],
 270983: [18, 22, 11, 9],
 270982: [-1, -6, -2, -3],
 270980: [15, 36, 14, 6],
 270977: [3, 14, 6,

In [14]:
id_language_df = df[['id', 'language']]

In [15]:
user_language_info = {}

# Dictionary for every unique user_id and the language of the client associated with the user_id
for index, row in id_language_df.iterrows():
    try:
        if(row['id'] not in user_language_info):
            user_language_info[row['id']] = [row['language']]
        else:
            languages = user_language_info[row['id']]
            result = set(languages.extend([row['language']]))
            user_info[row['id']] = list(result)
    except:
        continue
        
for key, value in user_language_info.items():
    languages = set(value)
    user_language_info[key] = languages

In [16]:
user_language_info

{271559: {'Spanish'},
 271558: {'Spanish'},
 271537: {'English'},
 271160: {'Portuguese'},
 271153: {'Spanish'},
 271152: {'English'},
 271149: {'English'},
 271148: {'English'},
 271147: {'Spanish'},
 271146: {'English'},
 271145: {'Spanish'},
 271127: {'Spanish'},
 271119: {'Spanish'},
 271117: {'English'},
 271113: {'Spanish'},
 271110: {'Spanish'},
 271105: {'Spanish'},
 271103: {'English'},
 271099: {'Spanish'},
 271096: {'Spanish'},
 271094: {'English'},
 271089: {'English'},
 271082: {'English'},
 271081: {'Spanish'},
 271080: {'Spanish'},
 271077: {'Portuguese'},
 271072: {'Spanish'},
 271071: {'Spanish'},
 271068: {'Portuguese'},
 271066: {'Spanish'},
 271065: {'Spanish'},
 271063: {'Spanish'},
 270993: {'Spanish'},
 270992: {'Spanish'},
 270991: {'Spanish'},
 270988: {'Spanish'},
 270983: {'Spanish'},
 270982: {'Spanish'},
 270980: {'English'},
 270977: {'Spanish'},
 270976: {'Spanish'},
 270974: {'Spanish'},
 270971: {'Spanish'},
 270969: {'Spanish'},
 270966: {'Spanish'},
 

In [17]:
client_df = df['client_id']

In [18]:
client_country_df = pd.read_csv('client_info.csv')

In [19]:
client_country_df.head()

,#,Client ID,Client Name,Distributor ID,Distributor Name,Country,Language,Notes,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,73.0,Focus Your Mind,20.0,Focus Your Mind - Paid,Colombia,Spanish,NaN,NaN,Please also update the client profile using LMS,NaN,NaN,NaN
1,2,123.0,Idencorp virtual store,29.0,Idencorp (paid account),Colombia,Spanish,NaN,NaN,LMS-> Profile->EditProfile->Country->Save,NaN,NaN,NaN
2,3,274.0,LANGUAGE SKILLS LIMA,20.0,Focus Your Mind - Paid,Peru,Spanish,NaN,NaN,[],NaN,NaN,NaN
3,4,291.0,UPiloto,36.0,Berlitz (paid account),Colombia,Spanish,NaN,NaN,NaN,NaN,NaN,NaN
4,5,394.0,CIES Client account,60.0,CIES (paid account),Colombia,Spanish,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import math 

# Client countries dicitionary of languages for every client_id
client_countries_lang = {}

for index, row in client_country_df.iterrows():
    if(row['Client ID'] == 1214):
        client_countries_lang[row['Client ID']] = ['Thailand', 'Thai']
    elif(row['Client ID'] == 1254):
        client_countries_lang[row['Client ID']] = ['Ecuador', 'Spanish']
    elif(row['Client ID'] == 1395):
        client_countries_lang[row['Client ID']] = ['Colombia', 'Spanish']
    elif(math.isnan(row['Client ID'])):
        continue
    else:
        client_countries_lang[row['Client ID']] = [row['Country'], row['Language']]

In [21]:
client_countries_lang

{73.0: ['Colombia', 'Spanish'],
 123.0: ['Colombia', 'Spanish'],
 274.0: ['Peru', 'Spanish'],
 291.0: ['Colombia', 'Spanish'],
 394.0: ['Colombia', 'Spanish'],
 459.0: ['Chile', 'Spanish'],
 597.0: ['Colombia', 'Spanish'],
 615.0: ['Mexico', 'Spanish'],
 642.0: ['Peru', 'Spanish'],
 726.0: ['Peru', 'Spanish'],
 729.0: ['Colombia', 'Spanish'],
 750.0: ['Israel', 'Hebrew'],
 752.0: ['Colombia', 'Spanish'],
 876.0: ['Brazil', 'Portugues'],
 883.0: ['Brazil', 'Portugues'],
 1068.0: ['Mexico', 'Spanish'],
 1115.0: ['Colombia', 'Spanish'],
 1214.0: ['Thailand', 'Thai'],
 1254.0: ['Ecuador', 'Spanish'],
 1255.0: ['Mexico', 'Spanish'],
 1371.0: ['Colombia', 'Spanish'],
 1392.0: ['Colombia', 'Spanish'],
 1395.0: ['Colombia', 'Spanish']}

In [22]:
# import session_info
# session_info.show()